In [1]:
import sys
sys.path.append("/home/graeme")
sys.path.append("/home/graeme/GBEnergyDataManager/GBEnergyDataManager")
%load_ext autoreload
%autoreload 2

In [3]:
import psycopg2
import pandas as pd
import datetime as dt
import gzip
import numpy as np
import pytz
import math
from tqdm import tqdm_notebook # pip install tqdm
import xml.etree.cElementTree as et
import requests
import xmltodict # pip install xmltodict
from GBEnergyDataManager.GBEnergyDataManager.settings import DATABASES, NETA_USER, NETA_PWD, NETA_BMU_LIST_URL
from GBEnergyDataManager.GBEnergyDataManager.utils import dt_to_sp, sp_to_dt

In [ ]:
# set period to extract data
start_date, end_date = dt.date(2017,1,1), dt.date(2019,7,17)

# create list of clock change dates
transition_days = [dt.date(x.year, x.month, x.day) for x in pytz.timezone('Europe/London')._utc_transition_times]

# set up blank dataframe, with settlement periods as index
blank_df = pd.DataFrame(columns = ['sd', 'sp'])
curr_date = start_date
while curr_date <= end_date:
    curr_sp = 1
    if curr_date in transition_days[::2]:
        max_sp = 46
    elif curr_date in transition_days[1::2]:
        max_sp = 50
    else:
        max_sp = 48
    while curr_sp <= max_sp:
        new_data = pd.DataFrame([[curr_date, curr_sp]], columns = ['sd', 'sp'])
        blank_df = blank_df.append(new_data)
        curr_sp += 1
    curr_date += dt.timedelta(days=1)  
blank_df = blank_df.set_index(['sd', 'sp'])

In [ ]:
#P114_df = pd.read_csv('/home/graeme/Downloads/C0421_scotland_df.csv')

In [34]:
new_bmus = pd.read_excel('/home/graeme/GBEnergyDataManager/Physical/unit_data/reg_bm_units_190724.xlsx',
                        skiprows = 2)
old_bmus = pd.read_csv('/home/graeme/GBEnergyDataManager/Physical/unit_data/reg_bm_units.csv')
new_bmus[(new_bmus['BM Unit ID'].str.startswith('E') | (new_bmus['BM Unit ID'].str.startswith('T'))) 
       & (~new_bmus['BM Unit ID'].isin(old_bmus['BM Unit ID']))]

,BM Unit ID,BMU Name,Party Name,Party ID,BMU Type,NGC BMU Name,GSP Group Id,GSP Group Name,Trading Unit Name,Prod/Cons Flag,...,NWDBMCAIC,WDBMCAEC,NWDBMCAEC,Exempt Export Flag,Base TU Flag,FPN Flag,Interconnector Id,Effective From,Manual Credit Qualifying Flag,Credit Qualifying Status
3796,E_CHESP-1,CHESHIRE PS - OCGT,RWE Generation UK plc,INNOGY01,E,NaN,_D,Merseyside and North Wales,DEFAULT__D,P,...,-0.094,4.306,4.306,T,T,F,NaN,2019-06-01,F,F
3797,E_CHESP-2,CHESHIRE PS - East,RWE Generation UK plc,INNOGY01,E,NaN,_D,Merseyside and North Wales,DEFAULT__D,P,...,-0.094,0.749,0.749,T,T,F,NaN,2019-06-01,F,F
3798,E_CHESP-3,CHESHIRE PS - West,RWE Generation UK plc,INNOGY01,E,NaN,_D,Merseyside and North Wales,DEFAULT__D,P,...,-0.094,1.310,1.310,T,T,F,NaN,2019-06-01,F,F
3808,E_CRSSB-1,Carnegie Road 1,Orsted ESS Mersey Limited,DONG014,E,CRSSB-1,_D,Merseyside and North Wales,DEFAULT__D,P,...,-8.000,8.000,8.000,T,T,T,NaN,2019-06-01,F,T
4327,T_EAAO-1,East Anglia One Offshore 1,East Anglia One Limited,EAOWIND,T,EAAO-1,NaN,NaN,NaN,NaN,...,-1.200,136.000,136.000,F,F,T,NaN,2019-07-08,F,T
4328,T_EAAO-2,East Anglia One Offshore 2,East Anglia One Limited,EAOWIND,T,EAAO-2,NaN,NaN,NaN,NaN,...,-1.200,136.000,136.000,F,F,T,NaN,2019-07-08,F,T
4474,T_NWCRS-1,New Cross 1,EDF Energy Customers Ltd,LENCO,T,NaN,NaN,NaN,NaN,NaN,...,-1.228,0.000,0.000,F,F,F,NaN,2019-06-01,F,F
4475,T_NWCRS-2,New Cross 2,EDF Energy Customers Ltd,LENCO,T,NaN,NaN,NaN,NaN,NaN,...,-1.228,0.000,0.000,F,F,F,NaN,2019-06-01,F,F
4527,T_SEEL-1,Spalding Energy Expansion Ltd,Spalding Energy Expansion Ltd,SPALDI,T,SEEL-1,NaN,NaN,NaN,NaN,...,-116.994,119.600,119.600,F,F,T,NaN,2019-06-19,F,T
4552,T_STBRD-1,Stalybridge 1,EDF Energy Customers Ltd,LENCO,T,NaN,NaN,NaN,NaN,NaN,...,-2.047,0.000,0.000,F,F,F,NaN,2019-07-18,F,F


In [31]:
len(new_bmus)

4611

In [15]:
# get list of BMUs from NETA website
#bmus = pd.read_csv('/home/graeme/GBEnergyDataManager/Physical/unit_data/reg_bm_units.csv')

response = requests.get(NETA_BMU_LIST_URL.format(NETA_USER, NETA_PWD))
#parsedXML = et.fromstring(response.content)
xml_dict = xmltodict.parse(response.content)
neta_bmus = pd.DataFrame.from_dict(xml_dict['response']['bmunit'])
neta_bmus

,@bmunitid,@ngcbmunitid,@name,@fueltype
0,E_MARK-1,MARK-1,Rothes Bio-Plant CHP 1,BONE
1,E_MARK-2,MARK-2,Rothes Bio-Plant CHP 2,BONE
2,I_IBD-BARC1,IBD-BARC1,Britned Interconnector (Demand) - Barclays Ban...,BRITNEDICD
3,I_IBD-BPGE1,IBD-BPGE1,Britned Interconnector (Demand) - BP Gas Marke...,BRITNEDICD
4,I_IBD-BRTN1,IBD-BRTN1,Britned Interconnector (Demand) - BritNed Deve...,BRITNEDICD
5,I_IBD-CENT1,IBD-CENT1,Britned Interconnector (Demand) - Centrica Ene...,BRITNEDICD
6,I_IBD-CGML1,IBD-CGML1,Britned Interconnector (Demand) - Citigroup Gl...,BRITNEDICD
7,I_IBD-DSKC1,IBD-DSKC1,Britned Interconnector (Demand) - Danske Commo...,BRITNEDICD
8,I_IBD-DBNK1,IBD-DBNK1,Britned Interconnector (Demand) - Deutsche Ban...,BRITNEDICD
9,I_IBD-DNGN1,IBD-DNGN1,Britned Interconnector (Demand) - Dong Energy ...,BRITNEDICD


In [19]:
conn = psycopg2.connect("dbname='ElexonData' user={} host={} password={}".format(DATABASES['default']['USER'],
                                                                                DATABASES['default']['HOST'],
                                                                                DATABASES['default']['PASSWORD']))
cur = conn.cursor()

In [22]:
bmus_query = 'SELECT * from bmra_bmu;'
sql_df = pd.read_sql(bmus_query, conn)
sql_df[(sql_df['id'].str.startswith('T')) 
       & (~sql_df['id'].isin(neta_bmus['@bmunitid'].unique()))]

,id,name
1025,T_WBURB-43,
1029,T_WBURB-41,
1038,T_PNYCB-1,
1040,T_AFTOW-1,
1041,T_ABRBO-1,
1043,T_STLGW-3,
1044,T_STLGW-2,
1045,T_STLGW-1,
1047,T_WHIHW-1,
1048,T_JGPR-1,


In [ ]:
gsp_groups = ['SCOTLAND', 'NORTH of SCOTLAND GSP', 'SOUTH of SCOTLAND GSP']
bmu_ids = list(bmus[bmus['GSP Group Name'].isin(gsp_groups)][~bmus['BM Unit ID'].str.contains('__')]['BM Unit ID'])
for bmu_id in bmu_ids:
    blank_df[bmu_id] = np.nan

In [ ]:
combined_BAVs = blank_df.copy()

bav_query = 'SELECT sd, sp, sum(bmra_boav.bv) \
FROM public.bmra_boav \
where bmra_boav.bmu_id=\'%s\' \
group by sd, sp \
order by sd, sp'

for bmu_id in bmu_ids:
    sql_df = pd.read_sql(bav_query % bmu_id, conn)
    sql_df = sql_df.set_index(['sd', 'sp'])
    combined_BAVs[bmu_id] = sql_df

#combined_BAVs.to_csv('~/scotgov_analysis_190718/BAVs.csv')

In [ ]:
combined_OAVs = blank_df.copy()

oav_query = 'SELECT sd, sp, sum(bmra_boav.ov) \
FROM public.bmra_boav \
where bmra_boav.bmu_id=\'%s\' \
group by sd, sp \
order by sd, sp'

for bmu_id in bmu_ids:
    sql_df = pd.read_sql(oav_query % bmu_id, conn)
    sql_df = sql_df.set_index(['sd', 'sp'])
    combined_OAVs[bmu_id] = sql_df

combined_OAVs.to_csv('~/scotgov_analysis_190718/OAVs.csv')

In [ ]:
combined_FPNs = blank_df.copy()

query = 'SELECT sd, sp, bmra_fpnlevel.ts, vp \
FROM public.bmra_fpnlevel \
left join public.bmra_fpn \
on bmra_fpnlevel.fpn_id = bmra_fpn.id \
where bmra_fpn.bmu_id=\'%s\' \
order by sd, sp, bmra_fpnlevel.ts'

for bmu_id in bmu_ids:
    bmu_fpns = []
    print(bmu_id)
    df = pd.read_sql(query % bmu_id, conn)
    prev_SD, prev_SP, prev_VP, prev_TS = None, None, None, None
    total_mwh = 0.0
    for key, values in df.iterrows():
        SD, SP, TS, VP = values['sd'], values['sp'], values['ts'], values['vp']
        if SD==prev_SD and SP==prev_SP: 
            time_diff = (TS - prev_TS).total_seconds() / 3600.0
            total_mwh += (prev_VP * time_diff + (0.5 * (VP - prev_VP) * time_diff))
        elif prev_SD is not None and prev_SP is not None:
            bmu_fpns.append([dt.date(prev_SD.year,prev_SD.month,prev_SD.day),prev_SP,total_mwh])
            total_mwh = 0.0
        prev_SD, prev_SP, prev_VP, prev_TS = SD, SP, VP, TS
    new_data = pd.DataFrame(bmu_fpns, columns=['sd', 'sp', bmu_id])
    new_data = new_data.set_index(['sd', 'sp'])
    combined_FPNs.update(new_data)
    
combined_FPNs.to_csv('~/scotgov_analysis_190718/FPNs.csv')

In [ ]:
combined_FPNs[combined_FPNs.sum(axis=1) == 0]

In [ ]:
combined_MELs = blank_df.copy()

query = 'SELECT bmra_mel.ts as tsr, sd, sp, bmra_mellevel.ts, ve \
FROM public.bmra_mellevel \
left join public.bmra_mel \
on bmra_mellevel.mel_id = bmra_mel.id \
where bmra_mel.bmu_id=\'%s\' \
order by sd, sp, bmra_mel.ts desc, bmra_mellevel.ts'

for bmu_id in tqdm_notebook(bmu_ids[:10], desc='total'):
    bmu_mels = []
    #print(bmu_id)
    df = pd.read_sql(query % bmu_id, conn)
    prev_SD, prev_SP, prev_VP, prev_TS, prev_TSR, last_msg_time = None, None, None, None, None, None
    total_mwh, total_time = 0.0, 0.0
    for key, values in tqdm_notebook(df.iterrows(), total=df.shape[0], desc=bmu_id):
        SD, SP, TS, VP, TSR = values['sd'], values['sp'], values['ts'], values['ve'], values['tsr']
        if SD==prev_SD and SP==prev_SP: 
            if TSR==last_msg_time: 
                time_diff = (TS - prev_TS).total_seconds() / 3600.0
                total_mwh += (prev_VP * time_diff + (0.5 * (VP - prev_VP) * time_diff))
                total_time += time_diff
        elif prev_SD is not None and prev_SP is not None:
            bmu_mels.append([dt.date(prev_SD.year,prev_SD.month,prev_SD.day),prev_SP,total_mwh])
            #if not math.isclose(0.5, total_time, rel_tol=1e-5):
            #    print(prev_SD, prev_SP)
            #    print('total time duration: {:+06.2f}'.format(total_time))
            total_mwh, total_time = 0.0, 0.0
            last_msg_time = values['tsr']
        prev_SD, prev_SP, prev_VP, prev_TS = SD, SP, VP, TS
    new_data = pd.DataFrame(bmu_mels, columns=['sd', 'sp', bmu_id])
    new_data = new_data.set_index(['sd', 'sp'])
    combined_MELs.update(new_data)
    

#combined_MELs.to_csv('~/scotgov_analysis_190718/MELs.csv')

In [ ]:
combined_MELs[combined_MELs.sum(axis=1) == 0]

In [ ]:
# search BOAV data to figure out what's happening with Peterhead values
filenames = df.get_tibco_daily_filenames(dt.date(2017,3,20), dt.date(2017,3,20))

for filename in filenames:
    file = gzip.open('/home/graeme/ElexonData/Unprocessed/' + filename, 'rb')
    file_content = file.read().decode('utf-8', 'ignore')
    file_rows = [entry for entry in file_content.split('}')]
    count = 0
    for message in file_rows:
        if len(message.strip()) > 0:
            message_dict = uf.message_to_dict(message+'}')
            if message_dict['message_subtype'] == 'BOAV':
                if message_dict['bmu_id'] in ['T_PEHE-1'] and message_dict['SP']==17:
                    print(message_dict)      
            count += 1
    file.close()

In [ ]:
# calculate cashflow by settlement period
combined_cashflows = blank_df.copy()

cashflows_query = 'SELECT sd, sp, sum(oc)-sum(bc) as cashflow \
FROM public.bmra_ebocf \
where bmu_id=\'%s\' \
group by sd, sp \
order by sd, sp;'

for bmu_id in bmu_ids:
    sql_df = pd.read_sql(cashflows_query % (bmu_id), conn)
    sql_df = sql_df.set_index(['sd', 'sp'])
    sql_df.rename(columns={'cashflow': bmu_id}, inplace=True)
    combined_cashflows.update(sql_df)

combined_cashflows.to_csv('~/scotgov_analysis_190718/cashflows.csv')

In [ ]:
# retrieve periods where acceptances 'potentially impacted by transmission constraints'
transmission_constraints = blank_df.copy()

trans_query = 'SELECT ts \
FROM public.bmra_boalf \
where bmu_id=\'%s\' \
and so=true \
order by ta;'

for bmu_id in bmu_ids:
    sql_df = pd.read_sql(trans_query % (bmu_id), conn)
    new_data = None
    for ts in sql_df['ts'].dt.tz_localize(tz='Europe/London').values:
        sd, sp = dt_to_sp(pd.Timestamp(ts))
        if new_data is None:
            new_data = np.array([[sd, sp, 1]])
        else:
            np.append(new_data,[sd, sp, 1])
    new_data_df = pd.DataFrame(new_data, columns=['sd', 'sp', bmu_id])
    new_data_df = new_data_df.set_index(['sd', 'sp'])
    transmission_constraints.update(new_data)
    

In [ ]:
bmus['GSP Group Name'].unique()

In [ ]:
bmus_query = 'SELECT * from bmra_bmu;'
sql_df[(sql_df['id'].str.startswith('E')) 
       & (~sql_df['id'].isin(bmus['BM Unit ID'].unique()))]

In [ ]:
bmus['BM Unit ID'].unique()